In [1]:
import matplotlib.pyplot as plt          
from torch_geometric.utils.convert import to_networkx
import torch
from torch_geometric.data import Dataset
from torch_geometric.data import download_url
import os
from torch_geometric.io import read_planetoid_data
from torch_geometric.datasets import Planetoid
import numpy as np
from torch_geometric.data import Data
from torch.nn import Linear
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv,SAGEConv,GATConv
import os.path as osp   
from torch_geometric.nn import global_mean_pool
import scipy.sparse as sp
import torch_geometric.nn as pyg_nn
from torch_geometric.data import DataLoader
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import matthews_corrcoef
warnings.filterwarnings("ignore", category=Warning)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tqdm import tqdm
import torch.nn as nn
from config1 import config_runtime
from collections import Counter
from sklearn.metrics import confusion_matrix
import pandas as pd
import shutil 
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict

/home/yanzh/anaconda3/envs/glydeep/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MyOwnDataset2(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        

        # 初始化方法，继承自Dataset类，传入根目录、变换和预处理变换
    # 返回数据集源文件名，告诉原始的数据集存放在哪个文件夹下面，如果数据集已经存放进去了，那么就会直接从raw文件夹中读取。
    @property
    def raw_file_names(self):
        # pass # 不能使用pass，会报join() argument must be str or bytes, not 'NoneType'错误
        return []

    # 首先寻找processed_paths[0]路径下的文件名也就是之前process方法保存的文件名
    @property
    def processed_file_names(self):
        return ['datas0.pt','datas1.pt']

    # 用于从网上下载数据集，下载原始数据到指定的文件夹下，自己的数据集可以跳过
    def download(self):
        pass
    def process(self):
        #
        data_list=[]
        for i in range(1):
            path_list=os.listdir(file1)
            path_list.sort()
            npz=path_list[i]
            #print(npz)
            #pdb2=path_list[i]
            aa=os.path.join(file1,npz)
    
            fileload=np.load(aa)  #读取npz文件
            key=list(fileload.keys()) #获取npz里面的键，由一个个字典组成
            #print(key)

            node=fileload['H'] #获取节点以及节点特征，数据类型为numpy.ndarray
            #node = torch.tensor(x) #数据类型为torch.Tensor
            #print(np.shape(node))

            adj=fileload['A1'] #data.edge_index节点和边的邻接矩阵
            #adj2=fileload['A2'] #data.edge_attr: 边属性
    

            #邻接矩阵转换成COO稀疏矩阵及转换  
            edge_index_temp = sp.coo_matrix(adj)  
            #print('edge_index_temp为：')
            #print(np.shape(edge_index_temp))
            tar=fileload['T']
    
    
            indices = np.vstack((edge_index_temp.row, edge_index_temp.col))
            edge_index = torch.LongTensor(indices)
            #print(np.shape(edge_index))
            #节点及节点特征数据转换
            x = node
            #print(x)
            #x = x.squeeze(0)
            x = torch.FloatTensor(x)
            #print(np.shape(x))
            #print(tar)
            a=tar
            y=torch.LongTensor(a)
            #print(y)
    
            #edge_attr=adj2

            #构建数据集:为一张图，节点数量，节点特征，Coo稀疏矩阵的边(邻接矩阵)，边的特征矩阵,一个图一个标签
            data=Data(x=x, edge_index=edge_index,y=y)
            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            torch.save(data,osp.join(self.processed_dir,'datas{}.pt'.format(i)))
    
    def len(self):
        return 1

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'datas{}.pt'.format(idx)))
        return data
    



class ImprovedTripleGraphModel(nn.Module):
    def __init__(self, num_node_features, num_output_features=2, dropout_rate=0.3):
        super(ImprovedTripleGraphModel, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)

        # 定义三层GATConv用于B链图
        self.conv_1 = SAGEConv(num_node_features, 512)
        self.conv_2 = SAGEConv(512, 1024)
        self.conv_3 = SAGEConv(1024, 2)

        # 最终层（不应用sigmoid）
        self.fc1 = nn.Linear(256, 512)  # 3个64维的特征向量融合后的维度
        self.fc2 = nn.Linear(512, num_output_features)  # 输出层
        
        
    def forward(self, x,edge_index, batch):
        # 处理B链图
        
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv_1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv_2(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv_3(x,edge_index)
        x = F.relu(x)
        return F.softmax(x, dim=1)

In [3]:
model_path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/model/model1/55_model.pt'
model=ImprovedTripleGraphModel(1310)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)
model.eval()
device=config_runtime['device']
model.to(device)

ImprovedTripleGraphModel(
  (dropout): Dropout(p=0.3, inplace=False)
  (conv_1): SAGEConv(1310, 512)
  (conv_2): SAGEConv(512, 1024)
  (conv_3): SAGEConv(1024, 2)
  (fc1): Linear(in_features=256, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [4]:
def calculate_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    auc=roc_auc_score(true_labels, predicted_labels)
    mcc=matthews_corrcoef(true_labels, predicted_labels)
    return accuracy, precision, recall, f1 ,mcc

In [5]:
def save_color(input_pdb,output_pdb2,new_resiude):   #告诉正样本
    
    with open(input_pdb,'r') as infile,open(output_pdb2,'w') as outfile:
        for line in infile:
            if line.startswith('ATOM'):
                atom_name = line[12:16].strip()  
                residue_name = line[17:20].strip()  
                chain_id = line[21].strip()  
                residue_number = (line[22:27].strip())  
                x = float(line[30:38].strip())  
                y = float(line[38:46].strip())  
                z = float(line[46:54].strip()) 
                line2=line[60:66].strip()
                if (chain_id,residue_number,residue_name) in new_resiude:

                    b_factor_value = 1
                else:
                    b_factor_value = 0
                        # 修改 B-factor 列（第 61-66 列）
                line = line[:60] + f"{b_factor_value:6.2f}" + line[66:]
                
            outfile.write(line)  

In [6]:
def calculate_pos_or_neg(resiude,input_pdb):
    resiude_list=[]
    for i in resiude:
        chain=i[0]
        residueid=i[1]
        residuename=i[2]
        with open(input_pdb,'r') as infile:
            for line in infile:
                if line.startswith('ATOM'):
                    chain_id = line[21]
                    residue_id = int((line[22:26]))  
                    x = float(line[30:38])
                    y = float(line[38:46])
                    atom_type = line[13:16].strip()
                    z = float(line[46:54])
                    residue_id2 = str(line[22:27]).strip() 
                    residue_type = line[17:20] 
                    if chain_id == chain and residue_id2 == residueid and residue_type == residuename:
                        resiude_list.append([x,y,z,chain_id,residue_id2,residue_type,atom_type])
                
    return resiude_list

In [7]:
def calculate_pos_or_neg_A(resiude,input_pdb,A_chain):
    resiude_list=[]
    for i in resiude:
        chain=i[0]
        residueid=i[1]
        residuename=i[2]
        if chain != A_chain:
            continue
        with open(input_pdb,'r') as infile:
            for line in infile:
                if line.startswith('ATOM'):
                    chain_id = line[21]
                    residue_id = int((line[22:26]))  
                    x = float(line[30:38])
                    y = float(line[38:46])
                    atom_type = line[13:16].strip()
                    z = float(line[46:54])
                    residue_id2 = str(line[22:27]).strip() 
                    residue_type = line[17:20] 
                    if chain_id == chain and residue_id2 == residueid and residue_type == residuename:
                        resiude_list.append([x,y,z,chain_id,residue_id2,residue_type,atom_type])
                
    return resiude_list

In [8]:
def calculate_pos_or_neg_B(resiude,input_pdb,B_chain):
    resiude_list=[]
    for i in resiude:
        chain=i[0]
        residueid=i[1]
        residuename=i[2]
        if chain != B_chain:
            continue
        with open(input_pdb,'r') as infile:
            for line in infile:
                if line.startswith('ATOM'):
                    chain_id = line[21]
                    residue_id = int((line[22:26]))  
                    x = float(line[30:38])
                    y = float(line[38:46])
                    atom_type = line[13:16].strip()
                    z = float(line[46:54])
                    residue_id2 = str(line[22:27]).strip() 
                    residue_type = line[17:20] 
                    if chain_id == chain and residue_id2 == residueid and residue_type == residuename:
                        resiude_list.append([x,y,z,chain_id,residue_id2,residue_type,atom_type])
                
    return resiude_list

In [9]:
def get_pos_jiechu(pos_list,all_list,cut_off):
    rog_all_pos = defaultdict(set)
    cut_off=cut_off**2
    atom_pos=len(pos_list)
    atom_all=len(all_list)
    for i in range(atom_pos):   #pos数目
        atom_pos2=pos_list[i]
        pos_chain=atom_pos2[3]
        pos_resid=atom_pos2[4]
        pos_residue=atom_pos2[5]
        pos_atom=atom_pos2[6]
        for j in range(atom_all):  #all数目
            atom_all2=all_list[j]
            all_chain=atom_all2[3]
            all_resid=atom_all2[4]
            all_residue=atom_all2[5]
            all_atom=atom_all2[6]
            if all_chain ==pos_chain and all_resid == pos_resid and all_residue == pos_residue:
                continue
            distance=0
            for k in range(3):
                distance+=(atom_pos2[k]-atom_all2[k])**2
            if distance<=cut_off:
                rog_all_pos[(pos_chain,pos_resid,pos_residue)].add((all_chain,all_resid,all_residue))
    return rog_all_pos

def jilu_pos(rog_all_pos,neg_resiude):
    pos_return=[]
    for index,value in rog_all_pos.items():
        if all(item in neg_resiude for item in value):
            pos_return.append(index)
    return pos_return    

In [18]:
results = []
tru_all=[]
pres_all=[]
pres2_all=[]
cut_off=3

data=pd.read_csv('/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/test_data.csv')
for index,value in data.iterrows():
    name=value['Identifier']
    #if name !="1gua_B":
    #    continue 
    print(name)
    os.makedirs(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture6/change_pos/cut_off_{cut_off}/{name}', exist_ok=True)
    output_pdb2=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture6/change_pos/cut_off_{cut_off}/{name}/{name}_change.pdb'
    output_pdb=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture6/change_pos/cut_off_{cut_off}/{name}/{name}_color.pdb'
    if name[5] !="1":
        
        pdb_name2=name[0:4]
        pdb_name=f'{pdb_name2}.npz'    #自己改
        pdb_name3=f'{pdb_name2}_npz'
        A_chain='A'
        B_chain='B'
        # 源文件路径
        #source = f"/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/test/npz/{pdb_name}"
        #base_path='/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn'
        
        #os.makedirs(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/negative_knn/{knn_name}/{pdb_name2}')
        #os.makedirs(os.path.join(base_path2, pdb_name3), exist_ok=True)
        # 目标路径（可以是文件夹，也可以是完整的新文件路径）
        #destination = f"/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn/{pdb_name2}/{pdb_name3}/{pdb_name}"
        # 移动文件
        #shutil.copy(source, destination)
        file1=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn_10_6/{pdb_name2}/{pdb_name3}'
        dataset_test=MyOwnDataset2(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn_10_6/{pdb_name2}/{pdb_name3}/{pdb_name2}_pt')
        preds=[]
        tru=[]
        test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)
        for data in test_loader:
                data=data.to(device)
                out = model(data.x, data.edge_index, data.batch)
                
                pred=out.argmax(dim=1)
                aa = data.y
                
                # 收集真实标签和预测标签
                preds.extend(pred.cpu().numpy())
                tru.extend(aa.cpu().numpy())
        rew_all_sasa_A=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/test/A/renew/all/sasa/{pdb_name2}_A.pdb'
        rew_all_sasa_B=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/test/B/renew/all/sasa/{pdb_name2}_B.pdb'
        
        all_A=[]
        with open(rew_all_sasa_A,'r') as rec_file:
                    line = rec_file.readlines()
                    for i in line:
                        if i[0:3] =='SEQ':
                            i=i.strip()
                            chain=i[4]
                            bb=i[12:15]         #残基名称
                            dd=i[6:11].strip()   #残基编号
                            cc=float(i[19:26])   #sasa
                        #print(bb,cc)            # 将文件中的SASA值存储到字典中  
                            all_A.append((chain,dd,bb))
        all_B=[]
        with open(rew_all_sasa_B,'r') as rec_file:
                    line = rec_file.readlines()
                    for i in line:
                        if i[0:3] =='SEQ':
                            i=i.strip()
                            chain=i[4]
                            bb=i[12:15]         #残基名称
                            dd=i[6:11].strip()   #残基编号
                            cc=float(i[19:26])   #sasa
                        #print(bb,cc)            # 将文件中的SASA值存储到字典中  
                            all_B.append((chain,dd,bb))
                            
        all2=all_A+all_B
        
        result_dict={value:item for item,value in zip(preds,all2)}
        pos_resiude=[]
        for key,value in result_dict.items():
            if str(value)=='1':
                pos_resiude.append(key)
        neg_resiude=[]
        for key,value in result_dict.items():
            if str(value)=='0':
                neg_resiude.append(key)
        input_pdb=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/chain-all/{pdb_name2}.ent.pdb'
        
        
    if name[5] =="1":
        B_chain=name[-3]
        A_chain=name[-1]
        pdb_name=f'{name}.npz'    #自己改
        pdb_name3=f'{name}_npz'
        pdb_name2=f'{name[0:4]}_1.pdb'
        file1=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn_10_6/{name}/{pdb_name3}'  
        #########################各个文件的npz目录同样在10_6中###########
        dataset_test=MyOwnDataset2(f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture5/knn_10_6/{name}/{pdb_name3}/{name}_pt')
        test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)
        preds=[]
        tru=[]
        
        B_chain=name[-3]
        A_chain=name[-1]
        for data in test_loader:
            data=data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            
            pred=out.argmax(dim=1)
            aa = data.y
                
                # 收集真实标签和预测标签
            preds.extend(pred.cpu().numpy())
            tru.extend(aa.cpu().numpy())
        
        rew_all_sasa_A=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/test/A/renew/all/sasa/{name}_{A_chain}.pdb'
        rew_all_sasa_B=f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture/test/B/renew/all/sasa/{name}_{B_chain}.pdb'
        all_A=[]
        
        with open(rew_all_sasa_A,'r') as rec_file:
                line = rec_file.readlines()
                for i in line:
                    if i[0:3] =='SEQ':
                        i=i.strip()
                        chain=i[4]
                        bb=i[12:15]         #残基名称
                        dd=i[6:11].strip()   #残基编号
                        cc=float(i[19:26])   #sasa
                    #print(bb,cc)            # 将文件中的SASA值存储到字典中  
                        all_A.append((chain,dd,bb))
        all_B=[]
        with open(rew_all_sasa_B,'r') as rec_file:
                line = rec_file.readlines()
                for i in line:
                    if i[0:3] =='SEQ':
                        i=i.strip()
                        chain=i[4]
                        bb=i[12:15]         #残基名称
                        dd=i[6:11].strip()   #残基编号
                        cc=float(i[19:26])   #sasa
                    #print(bb,cc)            # 将文件中的SASA值存储到字典中  
                        all_B.append((chain,dd,bb))
        all2=all_A+all_B
        
        result_dict={value:item for item,value in zip(preds,all2)}
        pos_resiude=[]
        for key,value in result_dict.items():
            if str(value)=='1':
                pos_resiude.append(key)
        neg_resiude=[]
        for key,value in result_dict.items():
            if str(value)=='0':
                neg_resiude.append(key)
        input_pdb=f'/data5_large/home/yanzh/deeplearn/PeSTo/data_my/gz_interaction_structure2/updata/{pdb_name2}'
        
    #pos_list=calculate_pos_or_neg(pos_resiude,input_pdb)
    pos_list_A=calculate_pos_or_neg_A(pos_resiude,input_pdb,A_chain)   #A链中的正样本
    pos_list_B=calculate_pos_or_neg_B(pos_resiude,input_pdb,B_chain)   #B链中的正样本
    neg_list=calculate_pos_or_neg(neg_resiude,input_pdb)
    #all_list=calculate_pos_or_neg(all2,input_pdb)
    all_list_A=calculate_pos_or_neg_A(all2,input_pdb,A_chain)    #A链中的所有样本
    all_list_B=calculate_pos_or_neg_B(all2,input_pdb,B_chain)    #B链中的所有样本
    rog_all_pos_A=get_pos_jiechu(pos_list_A,all_list_A,cut_off)
    rog_all_pos_B=get_pos_jiechu(pos_list_B,all_list_B,cut_off)
    #print(f'rog_all_pos_A',rog_all_pos_A)
    #print(f'rog_all_pos_B',rog_all_pos_B)
    merged_rog_all_pos = defaultdict(set)

# 添加 A 中的内容
    for key, value_set in rog_all_pos_A.items():
        merged_rog_all_pos[key].update(value_set)

# 添加 B 中的内容
    for key, value_set in rog_all_pos_B.items():
        merged_rog_all_pos[key].update(value_set)
    #print(f'rog_all_pos',merged_rog_all_pos)
    #print(len(rog_all_pos_A),len(rog_all_pos_B),len(merged_rog_all_pos))
    pos_return = jilu_pos(merged_rog_all_pos,neg_resiude)
    #rint('pos_return',pos_return)
    pos_diff = [item for item in pos_resiude if item not in pos_return]
    preds2 = [1 if item in pos_diff else 0 for item in all2]
    
    

    for i in tru:
        tru_all.append(i)
    for i in preds2:   #修改后的
        pres2_all.append(i)  #修改后的
    for i in preds:    #机器预测的
        pres_all.append(i)  #机器预测的
        
    save_color(input_pdb,output_pdb2,pos_diff)   #修改后
    save_color(input_pdb,output_pdb,pos_resiude)   #修改前
accuracy_new, precision_new, recall_new, f1_new ,mcc_new=calculate_metrics(tru_all,pres2_all)
accuracy, precision, recall, f1, mcc=calculate_metrics(tru_all,pres_all)
results.append([accuracy_new, precision_new, recall_new, f1_new ,mcc_new, accuracy, precision, recall, f1, mcc])
columns = ['accuracy_new', 'precision_new', 'recall_new', 'f1_new', 'mcc_new', 'accuracy', 'precision', 'recall', 'f1', 'mcc']
df = pd.DataFrame(results, columns=columns)
output_csv =f'/data5_large/home/yanzh/deeplearn/graph_residue/my_data/protein_new_3222/dataset_shuanglian_buchong/a_picture6/change_pos/cut_off_{cut_off}/{cut_off}_final.csv'
df.to_csv(output_csv, index=False)


8V9Q_1_A_D


Processing...
Done!


8V9Q_1_A_H


Processing...
Done!


4D0T_1_C_P


...

Processing...
Done!


4kdi_B


Processing...
Done!


5j8h_B


Processing...
Done!
